# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbesol.v0_4_1.oncvpsp3.standard.upf")
model = model_DFT(bulk(:Si); functionals=PBEsol(), pseudopotentials)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397859268244                   -0.90    5.2   36.7ms
  2   -8.400240798871       -2.62       -1.74    1.0   24.1ms
  3   -8.400406519292       -3.78       -2.97    1.5   47.4ms
  4   -8.400428702136       -4.65       -2.97    3.0   32.2ms
  5   -8.400428821648       -6.92       -3.06    1.0   24.6ms
  6   -8.400429019093       -6.70       -4.65    1.0   24.7ms
  7   -8.400429024004       -8.31       -4.46    2.2   29.9ms
  8   -8.400429024387       -9.42       -5.07    1.0   40.8ms
  9   -8.400429024418      -10.50       -6.70    1.0   25.3ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -8.397834962061                   -0.90           5.2    466ms
  2   -8.400384204413       -2.59       -1.79   0.80    2.0    503ms
  3   -8.400422950902       -4.41       -3.00   0.80    1.0    216ms
  4   -8.400428976763       -5.22       -3.44   0.80    2.8   28.6ms
  5   -8.400429021014       -7.35       -4.59   0.80    1.0   21.4ms
  6   -8.400429024396       -8.47       -6.30   0.80    2.5   29.0ms


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +1.168132159666                   -1.11    2.74s
  2   -1.676652370668        0.45       -0.67    136ms
  3   -3.615554832479        0.29       -0.39   56.6ms
  4   -4.542300966760       -0.03       -0.45   57.3ms
  5   -6.410491558129        0.27       -0.50   80.3ms
  6   -7.220813593200       -0.09       -0.72   56.8ms
  7   -7.983534505378       -0.12       -0.92   56.8ms
  8   -8.128555180332       -0.84       -1.36   42.4ms
  9   -8.215957183227       -1.06       -1.40   73.4ms
 10   -8.261188517463       -1.34       -1.83   56.7ms
 11   -8.322013342626       -1.22       -2.26   42.2ms
 12   -8.339558788369       -1.76       -2.14   45.8ms
 13   -8.367196469885       -1.56       -2.49   42.4ms
 14   -8.383600156894       -1.79       -2.39   42.2ms
 15   -8.396696212375       -1.88       -2.46   42.2ms
 16   -8.397361188884       -3.18       -2.66   46.2ms
 17   -8.39

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397858581254                   -0.90    5.5   41.5ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -8.400428852547                   -1.78    24.5s
  2   -8.400429024420       -6.76       -4.02    2.32s
  3   -8.400429024420      -14.45       -7.83    128ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.4523858411236823e-7
|ρ_newton - ρ_scfv| = 5.006558045494872e-6
|ρ_newton - ρ_dm|   = 9.76446280429483e-7
